In [ ]:
import os
os.environ['HF_TOKEN'] = '' # your Hugging Face token here

URL: https://huggingface.co/Dellalala1/LlaMA_3.2_1B_Fine-tuned 

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

In [ ]:
# Load model directly
from transformers import AutoModel
path="Dellalala1/LlaMA_3.2_1B_Fine-tuned" 
model = AutoModelForCausalLM.from_pretrained(path, dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(path)

In [8]:
prod_info=pd.read_csv('real_queries_extracted.csv') # change to result direct retrieval / recommendation
prod_info.head()

,rank,query,constraints,pid,title,summary,scores,product
0,1,I want a lipstick for dating,"{'brand': 'nicka k new york', 'price_min': 2.0...",B004T4V480,NICKA K LIPSTICK WITH VITAMIN E MOODY BLUE #990,"Lanolin causes chapped lips, not good for thos...","{'fused': 0.8061031699180601, 'ce': -5.9273662...","{'main_category': 'All Beauty', 'title': 'NICK..."
1,2,I want a lipstick for dating,"{'brand': 'nicka k new york', 'price_min': 2.0...",B004T4R4WU,NICKA K LIPSTICK WITH VITAMIN E AMETHYST #901,Great color lipstick.. I love this lipstick. I...,"{'fused': 0.7245547175407411, 'ce': -6.6143398...","{'main_category': 'All Beauty', 'title': 'NICK..."
2,3,I want a lipstick for dating,"{'brand': 'nicka k new york', 'price_min': 2.0...",B004TBEA84,NICKA K LIPSTICK WITH VITAMIN E RUSSET BROWN #705,"Love, love, love it!!!. This is the best lipst...","{'fused': 0.708895087242126, 'ce': -6.40550088...","{'main_category': 'All Beauty', 'title': 'NICK..."
3,4,I want a lipstick for dating,"{'brand': 'nicka k new york', 'price_min': 2.0...",B004TBIXK0,Nicka K Lipstick With Vitamin E (306-Black),"The back lipstick is nice, I like the color. T...","{'fused': 0.669598996639251, 'ce': -6.46536350...","{'main_category': 'All Beauty', 'title': 'Nick..."
4,5,I want a lipstick for dating,"{'brand': 'nicka k new york', 'price_min': 2.0...",B004T4MYDO,NICKA K LIPSTICK WITH VITAMIN E PINK TINT #921,Great color. I really love this color. I broug...,"{'fused': 0.666649997234344, 'ce': -6.55975246...","{'main_category': 'All Beauty', 'title': 'NICK..."


In [20]:
def generate_answer(question: str):
    prompt = f"### Instruction:\n{question}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1.1,           
            top_p=0.85,                 
            repetition_penalty=1.1,   # adjust parameters if needed 
            do_sample=True,            
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.split("### Response:")[-1].strip()
    return answer

In [16]:
def format_recommendation(rag_results):
    context = "Here are the top recommended products based on your query:\n\n"

    for idx, row in rag_results.iterrows():
        context += f"Product {idx + 1}:\n"
        context += f"- Name: {row['title']}\n"
        context += f"- Product ID: {row['pid']}\n"
        context += f"- Description: {row['summary']}\n"
        context += f"- Relevance Score: {row['scores']}\n"
        context += "\n"

    return context

In [26]:
def format_inquiry(rag_results):
    context = "Here are the product information based on your query:\n\n"

    for idx, row in rag_results.iterrows():
        context += f"Product {idx + 1}:\n"
        context += f"- Name: {row['title']}\n"
        context += f"- Product ID: {row['pid']}\n"
        context += f"- Description: {row['summary']}\n"
        context += f"- Relevance Score: {row['scores']}\n"
        context += "\n"

    return context

In [ ]:
def generate_answer_with_rag(user_query, rag_results=None, intent=None):
    
    if intent == "product recommendation": #
        product_context = format_recommendation(rag_results)
        enhanced_prompt = f"""You are a helpful, professional, and friendly customer service representative. Your goal is to assist customers efficiently while maintaining a warm, conversational tone.
        Based on the following product recommendations, provide a helpful response to the user's query.
        Product Recommendations:{product_context}
        User Query: {user_query}
        Please recommend the most suitable products and explain why they would be good choices. Be natural and professional in your response."""
        generated_text = generate_answer(enhanced_prompt)
    elif intent == "product inquiry":
        product_context = format_inquiry(rag_results)
        enhanced_prompt = f"""You are a helpful, professional, and friendly customer service representative. Your goal is to assist customers efficiently while maintaining a warm, conversational tone.
        Based on the following product information, provide a helpful response to the user's query. Be natural and professional in your response.
        Product Information:{product_context}
        User Query: {user_query}    """
        generated_text = generate_answer(enhanced_prompt)
    else:
        enhanced_prompt = f"""You are a helpful, professional, and friendly customer service representative. Your goal is to assist customers efficiently while maintaining a warm, conversational tone.
        Now provide a helpful response to the user's query. Be natural and professional in your response.
        User Query: {user_query}"""
        generated_text = generate_answer(enhanced_prompt)

    return generated_text


Recommendation example

In [14]:
eyeliner=prod_info.iloc[20:30,]
query1="Can you recommend a long-lasting eyeliner for everyday use?"

In [21]:
answer = generate_answer_with_rag(query1, eyeliner, intent="product recommendation")
print(answer)

Hi. A good choice would be MAC LACQUER LINER. You get a good quality liner but is still affordable. Also check out Huda Beauty, Urban Decay or NYX Pro All Day Pencils which all offer great eyeliner formulas that last all day. Thanks!


Product Inquiry example

In [22]:
hair_mask=prod_info.iloc[15:16,:]

In [30]:
query2='What type of hair is this hair product suitable for?'
answer = generate_answer_with_rag(query2, hair_mask,intent='product inquiry')
print(answer)

This conditioner should be okay for almost any type of hair. You might not want it on color-treated or very damaged hair because that can make it too greasy to put your hands through. But if you have curly or wavy or straight hair then this will definitely do things up!


others

In [33]:
query3='When will this hair mask be back in stock?'
answer = generate_answer_with_rag(query3, rag_results=None,intent=None)
print(answer)

Hello there! We don't have it in stock at this time but we can notify you as soon as it becomes available again. You may wish to sign up for our e mail newsletter (at www.silkmaker.com) to stay updated on any of our products that may go back into inventory. Thanks for your interest!
